# Import


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install plotly_express

In [ ]:
!pip install geopandas

  Using cached geopandas-0.10.2-py2.py3-none-any.whl (1.0 MB)
  Using cached pyproj-3.2.1-cp37-cp37m-manylinux2010_x86_64.whl (6.3 MB)
  Using cached Fiona-1.8.20-cp37-cp37m-manylinux1_x86_64.whl (15.4 MB)


In [ ]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'

 trips_2019_1_10.csv   trips_2019_3_13.csv   trips_2019_5_13.csv
 trips_2019_1_11.csv   trips_2019_3_14.csv   trips_2019_5_14.csv
 trips_2019_1_12.csv   trips_2019_3_15.csv   trips_2019_5_15.csv
 trips_2019_1_13.csv   trips_2019_3_16.csv   trips_2019_5_16.csv
 trips_2019_1_14.csv   trips_2019_3_17.csv   trips_2019_5_17.csv
 trips_2019_1_15.csv   trips_2019_3_18.csv   trips_2019_5_18.csv
 trips_2019_1_16.csv   trips_2019_3_19.csv   trips_2019_5_19.csv
 trips_2019_1_17.csv   trips_2019_3_1.csv    trips_2019_5_1.csv
 trips_2019_1_18.csv   trips_2019_3_20.csv   trips_2019_5_20.csv
 trips_2019_1_19.csv   trips_2019_3_21.csv   trips_2019_5_21.csv
 trips_2019_1_1.csv    trips_2019_3_22.csv   trips_2019_5_22.csv
 trips_2019_1_21.csv   trips_2019_3_23.csv   trips_2019_5_23.csv
 trips_2019_1_22.csv   trips_2019_3_24.csv   trips_2019_5_24.csv
 trips_2019_1_23.csv   trips_2019_3_25.csv   trips_2019_5_25.csv
 trips_2019_1_24.csv   trips_2019_3_26.csv   trips_2019_5_26.csv
 trips_2019_1_25.csv   tri

In [ ]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'
export_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'

# Road data

In [ ]:
road_df = pd.read_csv(road_data_path+"/route_network_with_cluster.csv")
# road_df = pd.read_csv(road_data_path+"/roadLink_merge_roadDB.csv")
road_df.sort_values(by=['way_id', 'node_num'])
road_df

,way_id,node_num,highway,road_number,bridge,node_id,lat,lon,cluster,cluster_size,center_lat,center_lon
0,638011068,0,motorway_link,NaN,NaN,6583740783,14.982732,102.045890,0,156,14.984788,102.051257
1,701045037,19,motorway_link,NaN,viaduct,6583740783,14.982732,102.045890,0,156,14.984788,102.051257
2,638011068,1,motorway_link,NaN,NaN,8646920493,14.982586,102.044977,0,156,14.984788,102.051257
3,932823302,0,motorway_link,NaN,NaN,8646920493,14.982586,102.044977,0,156,14.984788,102.051257
4,932823304,0,motorway_link,NaN,NaN,8646920493,14.982586,102.044977,0,156,14.984788,102.051257
...,...,...,...,...,...,...,...,...,...,...,...,...
50525,940359905,39,tertiary,NaN,NaN,5688892503,15.557630,101.573573,8052,2,15.557630,101.573573
50526,941192524,0,tertiary,ชย.3006,yes,3634072858,15.782675,101.468727,8053,4,15.782620,101.468655
50527,941192525,156,tertiary,ชย.3006,NaN,3634072858,15.782675,101.468727,8053,4,15.782620,101.468655
50528,941192524,1,tertiary,ชย.3006,yes,3634072857,15.782565,101.468584,8053,4,15.782620,101.468655


# get array of road segment
make it become like this

[
  ((cluster_a.lat, cluster_b.lon),(cluster_b.lat, cluster_b.lon)),
  ((cluster_a.lat, cluster_b.lon),(cluster_b.lat, cluster_b.lon)),
  .....
]

drop ทุก way ที่มีแค่ 1 sp (cluster)

In [ ]:
not_use_way = []
for way_id in road_df.way_id.unique():
  if len(road_df[road_df['way_id'] == way_id].cluster.unique()) <= 1:
    not_use_way.append(way_id)

Drop all way_id that not the conection of two support point

In [ ]:
road_df2 = road_df[~(road_df['way_id'].isin(not_use_way))].sort_values(by=['way_id', 'node_num'])
road_df2

,way_id,node_num,highway,road_number,bridge,node_id,lat,lon,cluster,cluster_size,center_lat,center_lon
770,23730732,0,trunk,216.0,NaN,988611682,17.433014,102.804001,35,276,17.447362,102.793198
1046,23730732,21,trunk,216.0,NaN,6443458890,17.431047,102.809290,36,46,17.425429,102.811874
5020,23747253,0,tertiary,มค.3013,NaN,492040126,16.445877,103.031277,283,5,16.445831,103.031279
30694,23747253,174,tertiary,มค.3013,NaN,639543683,16.495235,103.050672,2864,6,16.496541,103.050376
30156,23747922,0,secondary,2335.0,NaN,257216756,14.531797,104.489508,2772,3,14.531797,104.489508
...,...,...,...,...,...,...,...,...,...,...,...,...
39275,983823827,15,secondary,2328,NaN,1137078608,14.488852,103.845738,4721,6,14.488899,103.845738
39276,983823829,0,secondary,2328,NaN,1137078608,14.488852,103.845738,4721,6,14.488899,103.845738
37618,983823829,29,secondary,2328,NaN,968277576,14.483397,103.795725,4346,3,14.483397,103.795725
39279,983823830,0,secondary,NaN,NaN,9097068327,14.488993,103.845739,4721,6,14.488899,103.845738


## clean way_id that have same cluster sequence

สร้าง set ว่าแต่ละ way มี sp อะไรบ้าง 

In [ ]:
cluster_set_list = []
for way_id in road_df2.way_id.unique():
  cluster_set = set(road_df2[road_df2['way_id'] == way_id]['cluster'].to_list())
  cluster_set_list.append(cluster_set)

หา way ที่มี set ของ sp เดียวกัน

In [ ]:
not_dup_set = []
duplicate_way = []
for way_id, cluster_set in zip(list(road_df2.way_id.unique()), cluster_set_list):
  if cluster_set not in not_dup_set:
    not_dup_set.append(cluster_set)
  else:
    duplicate_way.append(way_id)

Drop all way_id that have same support point sequence (เลือกแค่อันแรก)

In [ ]:
road_df3 = road_df2[~road_df2['way_id'].isin(duplicate_way)]
road_df3

,way_id,node_num,highway,road_number,bridge,node_id,lat,lon,cluster,cluster_size,center_lat,center_lon
770,23730732,0,trunk,216.0,NaN,988611682,17.433014,102.804001,35,276,17.447362,102.793198
1046,23730732,21,trunk,216.0,NaN,6443458890,17.431047,102.809290,36,46,17.425429,102.811874
5020,23747253,0,tertiary,มค.3013,NaN,492040126,16.445877,103.031277,283,5,16.445831,103.031279
30694,23747253,174,tertiary,มค.3013,NaN,639543683,16.495235,103.050672,2864,6,16.496541,103.050376
30156,23747922,0,secondary,2335.0,NaN,257216756,14.531797,104.489508,2772,3,14.531797,104.489508
...,...,...,...,...,...,...,...,...,...,...,...,...
39270,983757865,26,secondary,2040,NaN,9095940377,15.619644,103.239523,4719,3,15.619644,103.239523
31751,983823827,0,secondary,2328,NaN,1137078429,14.494259,103.869113,3069,12,14.494379,103.869163
39275,983823827,15,secondary,2328,NaN,1137078608,14.488852,103.845738,4721,6,14.488899,103.845738
39276,983823829,0,secondary,2328,NaN,1137078608,14.488852,103.845738,4721,6,14.488899,103.845738


Drop support point that duplicate in same way_id

In [ ]:
road_df3 = road_df3.drop_duplicates(subset=['way_id', 'cluster'])
road_df3

,way_id,node_num,highway,road_number,bridge,node_id,lat,lon,cluster,cluster_size,center_lat,center_lon
770,23730732,0,trunk,216.0,NaN,988611682,17.433014,102.804001,35,276,17.447362,102.793198
1046,23730732,21,trunk,216.0,NaN,6443458890,17.431047,102.809290,36,46,17.425429,102.811874
5020,23747253,0,tertiary,มค.3013,NaN,492040126,16.445877,103.031277,283,5,16.445831,103.031279
30694,23747253,174,tertiary,มค.3013,NaN,639543683,16.495235,103.050672,2864,6,16.496541,103.050376
30156,23747922,0,secondary,2335.0,NaN,257216756,14.531797,104.489508,2772,3,14.531797,104.489508
...,...,...,...,...,...,...,...,...,...,...,...,...
39270,983757865,26,secondary,2040,NaN,9095940377,15.619644,103.239523,4719,3,15.619644,103.239523
31751,983823827,0,secondary,2328,NaN,1137078429,14.494259,103.869113,3069,12,14.494379,103.869163
39275,983823827,15,secondary,2328,NaN,1137078608,14.488852,103.845738,4721,6,14.488899,103.845738
39276,983823829,0,secondary,2328,NaN,1137078608,14.488852,103.845738,4721,6,14.488899,103.845738


## get road segment

In [ ]:
cluster_coord = road_df3[['cluster', 'center_lat', 'center_lon']].drop_duplicates(subset=['cluster'])
cluster_coord.to_csv(road_data_path+"/cluster_coord.csv", index=False)
cluster_coord

,cluster,center_lat,center_lon
770,35,17.447362,102.793198
1046,36,17.425429,102.811874
5020,283,16.445831,103.031279
30694,2864,16.496541,103.050376
30156,2772,14.531797,104.489508
...,...,...,...
39230,4711,16.061351,103.300108
39235,4713,16.016124,103.315133
39238,4714,15.994301,103.320988
39266,4718,15.765669,103.319880


In [ ]:
way_segment_df = pd.DataFrame(columns=['way_id', 'seg_num', 'road_seg'])
for way_id in road_df3.way_id.unique():
  cluster_seq = road_df3[road_df3['way_id'] == way_id]['cluster'].to_list()
  seg_num = 0
  for road_segment in [(c1, c2) for c1, c2 in zip(cluster_seq[:-1], cluster_seq[1:])]:
    # make (1,2) and (2,1) the same thing
    if road_segment[0] > road_segment[1]:
      # print(road_segment, end='\t')
      road_segment = (road_segment[1], road_segment[0])
      # print(road_segment)
    way_segment_df.loc[len(way_segment_df.index)] = [way_id, seg_num, road_segment]
    seg_num = seg_num + 1
way_segment_df

,way_id,seg_num,road_seg
0,23730732,0,"(35, 36)"
1,23747253,0,"(283, 2864)"
2,23747922,0,"(2772, 2773)"
3,23747922,1,"(2773, 2774)"
4,23747922,2,"(2774, 2775)"
...,...,...,...
10630,983757864,0,"(4719, 4720)"
10631,983757864,1,"(3299, 4720)"
10632,983757865,0,"(4496, 4719)"
10633,983823827,0,"(3069, 4721)"


In [ ]:
way_segment_df.to_csv(road_data_path+"/way_and_road_segment.csv", index=False)

# Get nearest road segment of each trip observation 

## Read Gps data

In [ ]:
trips_df = pd.read_csv(trips_data_path+'/trips_2019_1_1.csv', parse_dates=['time_stamp'])
trips_df

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0
...,...,...,...,...,...,...,...
195857,2019-01-01 21:51:58,003000300000359769033400604,14.835272,101.550918,52,7.0,2098
195858,2019-01-01 21:53:56,003000300000359769033400604,14.846673,101.562518,61,7.0,2098
195859,2019-01-01 21:56:00,003000300000359769033400604,14.850647,101.572147,59,7.0,2098
195860,2019-01-01 21:57:55,003000300000359769033400604,14.844158,101.590907,60,7.0,2098


## Map trip to way sequence with ckd-tree

In [ ]:
trips_df = pd.read_csv(trips_data_path+'/trips_2019_1_1.csv', parse_dates=['time_stamp'])
trips_df[:5]

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0


In [ ]:
way_segment_df = pd.read_csv(road_data_path+"/way_and_road_segment.csv")
way_segment_df['road_seg'] = way_segment_df['road_seg'].map(lambda x: eval(x))
way_segment_df

,way_id,seg_num,road_seg
0,23730732,0,"(35, 36)"
1,23747253,0,"(283, 2864)"
2,23747922,0,"(2772, 2773)"
3,23747922,1,"(2773, 2774)"
4,23747922,2,"(2774, 2775)"
...,...,...,...
10630,983757864,0,"(4719, 4720)"
10631,983757864,1,"(3299, 4720)"
10632,983757865,0,"(4496, 4719)"
10633,983823827,0,"(3069, 4721)"


In [ ]:
from shapely.geometry import Point

cluster_coord = pd.read_csv(road_data_path+"/cluster_coord.csv")
cluster_coord = cluster_coord.set_index('cluster')

coords_list = []
for road_seg in list(way_segment_df.road_seg):
  p1 = cluster_coord.loc[road_seg[0]]
  p2 = cluster_coord.loc[road_seg[1]]
  p1 = Point(p1.center_lat, p1.center_lon)
  p2 = Point(p2.center_lat, p2.center_lon)
  coords_list.append((p1, p2))
coords_list[:5]

[(<shapely.geometry.point.Point at 0x7fdfc1d57f90>,
  <shapely.geometry.point.Point at 0x7fdfc1d57dd0>),
 (<shapely.geometry.point.Point at 0x7fdfc1d57e50>,
  <shapely.geometry.point.Point at 0x7fdfc1d57050>),
 (<shapely.geometry.point.Point at 0x7fdfc1d57fd0>,
  <shapely.geometry.point.Point at 0x7fdfc1d57f50>),
 (<shapely.geometry.point.Point at 0x7fdfc1d57ed0>,
  <shapely.geometry.point.Point at 0x7fdfc1d57f10>),
 (<shapely.geometry.point.Point at 0x7fdfc22cfa10>,
  <shapely.geometry.point.Point at 0x7fdfc22cfb10>)]

In [ ]:
way_segment_df["line_coords"] = coords_list
way_segment_df[:5]

,way_id,seg_num,road_seg,line_coords
0,23730732,0,"(35, 36)","(POINT (17.44736160760869 102.7931978869564), ..."
1,23747253,0,"(283, 2864)","(POINT (16.44583146 103.031279), POINT (16.496..."
2,23747922,0,"(2772, 2773)","(POINT (14.5317975 104.4895079), POINT (14.508..."
3,23747922,1,"(2773, 2774)","(POINT (14.508777 104.5188133), POINT (14.5095..."
4,23747922,2,"(2774, 2775)","(POINT (14.5095453 104.540198), POINT (14.4996..."


In [ ]:
def midpoint(line_coords):
  return (line_coords[0].x+line_coords[1].x)/2, (line_coords[0].y+line_coords[1].y)/2

way_segment_df["mid_points"] = way_segment_df["line_coords"].apply(lambda x: midpoint(x))
way_segment_df['mid_lat'] = way_segment_df['mid_points'].apply(lambda x: x[0])
way_segment_df['mid_lon'] = way_segment_df['mid_points'].apply(lambda x: x[1])
way_segment_df[:5]

,way_id,seg_num,road_seg,line_coords,mid_points,mid_lat,mid_lon
0,23730732,0,"(35, 36)","(POINT (17.44736160760869 102.7931978869564), ...","(17.43639512554348, 102.80253589130432)",17.436395,102.802536
1,23747253,0,"(283, 2864)","(POINT (16.44583146 103.031279), POINT (16.496...","(16.471186463333332, 103.04082748333333)",16.471186,103.040827
2,23747922,0,"(2772, 2773)","(POINT (14.5317975 104.4895079), POINT (14.508...","(14.520287249999999, 104.5041606)",14.520287,104.504161
3,23747922,1,"(2773, 2774)","(POINT (14.508777 104.5188133), POINT (14.5095...","(14.509161149999999, 104.52950565)",14.509161,104.529506
4,23747922,2,"(2774, 2775)","(POINT (14.5095453 104.540198), POINT (14.4996...","(14.504590525, 104.575137475)",14.504591,104.575137


FIXME: difference way but same road segment

In [ ]:
way_segment_df[way_segment_df.duplicated(subset=['mid_lat', 'mid_lon'], keep=False)].sort_values(by=['mid_lat', 'mid_lon'])[:5]

,way_id,seg_num,road_seg,line_coords,mid_points,mid_lat,mid_lon
2494,152621928,2,"(573, 593)","(POINT (14.3859538 101.86827805), POINT (14.40...","(14.393656816666665, 101.86549815833334)",14.393657,101.865498
3278,180247322,21,"(573, 593)","(POINT (14.3859538 101.86827805), POINT (14.40...","(14.393656816666665, 101.86549815833334)",14.393657,101.865498
2495,152621928,3,"(593, 595)","(POINT (14.40135983333333 101.8627182666667), ...","(14.410192641666665, 101.86245090833333)",14.410193,101.862451
3277,180247322,20,"(593, 595)","(POINT (14.40135983333333 101.8627182666667), ...","(14.410192641666665, 101.86245090833333)",14.410193,101.862451
2496,152621928,4,"(72, 595)","(POINT (14.4282542 101.86899526), POINT (14.41...","(14.423639824999999, 101.86558940500001)",14.423640,101.865589


FIX: lazy drop duplicate

In [ ]:
way_segment_drop_dup_df = way_segment_df[['mid_lat', 'mid_lon', 'way_id']].drop_duplicates(subset=['mid_lat', 'mid_lon'])
way_segment_drop_dup_df.to_csv(road_data_path + 'way_and_road_segment_lazy_drop.csv', index=False)

In [ ]:
import itertools
from operator import itemgetter

import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString

# from pd to gpd
trips_gdf = gpd.GeoDataFrame(
    trips_df, geometry=gpd.points_from_xy(trips_df.lat, trips_df.lon))

# create linestring of way_id
way_segment_drop_dup_gdf = gpd.GeoDataFrame(
  way_segment_drop_dup_df, geometry=gpd.points_from_xy(
      way_segment_drop_dup_df.mid_lat,
      way_segment_drop_dup_df.mid_lon
    )
  )

way_id_list = []
for way_id in pd.unique(way_segment_drop_dup_gdf.way_id):
  line = way_segment_drop_dup_gdf[way_segment_drop_dup_gdf.way_id == way_id]['geometry'].to_list()
  if len(line) < 2:
    line = line*2
  way_id_list.append([way_id, LineString(line)])
way_id_list

way_segment_drop_dup_gdf = gpd.GeoDataFrame(way_id_list, columns=['way_id', 'geometry'])

In [ ]:
trips_gdf[:5]

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,geometry
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0,POINT (16.482 104.050)
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0,POINT (16.491 104.052)
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0,POINT (16.503 104.055)
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0,POINT (16.514 104.056)
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0,POINT (16.525 104.056)


In [ ]:
way_segment_drop_dup_gdf[:5]

,way_id,geometry
0,23730732,"LINESTRING (17.436 102.803, 17.436 102.803)"
1,23747253,"LINESTRING (16.471 103.041, 16.471 103.041)"
2,23747922,"LINESTRING (14.520 104.504, 14.509 104.530, 14..."
3,23749257,"LINESTRING (17.172 104.099, 17.187 104.095)"
4,23749339,"LINESTRING (16.611 103.620, 16.634 103.628)"


In [ ]:
def ckdnearest(gdfA, gdfB, gdfB_cols=['way_id']):
    A = np.concatenate(
        [np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    B = [np.array(geom.coords) for geom in gdfB.geometry.to_list()]
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
    B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    idx = itemgetter(*idx)(B_ix)
    gdf = pd.concat(
        [gdfA, gdfB.loc[idx, gdfB_cols].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

trips_gdf_nearest_way = ckdnearest(trips_gdf, way_segment_drop_dup_gdf)

In [ ]:
trips_gdf_nearest_way

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,geometry,way_id,dist
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0,POINT (16.482 104.050),340294191,0.004308
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0,POINT (16.491 104.052),92193150,0.000498
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0,POINT (16.503 104.055),402348153,0.004740
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0,POINT (16.514 104.056),92193150,0.008590
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0,POINT (16.525 104.056),92193150,0.002386
...,...,...,...,...,...,...,...,...,...,...
195857,2019-01-01 21:51:58,003000300000359769033400604,14.835272,101.550918,52,7.0,2098,POINT (14.835 101.551),82374801,0.001614
195858,2019-01-01 21:53:56,003000300000359769033400604,14.846673,101.562518,61,7.0,2098,POINT (14.847 101.563),82374801,0.001829
195859,2019-01-01 21:56:00,003000300000359769033400604,14.850647,101.572147,59,7.0,2098,POINT (14.851 101.572),82374801,0.007574
195860,2019-01-01 21:57:55,003000300000359769033400604,14.844158,101.590907,60,7.0,2098,POINT (14.844 101.591),82374801,0.005860


In [ ]:
export_df = trips_gdf_nearest_way.drop(columns=['geometry'])
export_df.to_csv(export_data_path+'/trips_2019_1_1.csv', index=False)

## plot

In [ ]:
import random
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

trips_count = trips_gdf_nearest_way.trip_id.value_counts()

# sample_trip = random.sample(list(trips_count.index), 500)
sample_trip = random.sample(list(trips_count[trips_count > 2].index), 200)

fig = px.line_mapbox(
  trips_gdf_nearest_way[trips_gdf_nearest_way.trip_id.isin(sample_trip)], 
  lat="lat", 
  lon="lon", 
  color="unit_id", 
  hover_data= ['unit_id', 'trip_id','time_stamp', 'way_id'], 
  zoom=9, 
  width=1200, 
  height=800
)

fig.show()

In [ ]:
import random
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

print(trips_gdf_nearest_way[trips_gdf_nearest_way.trip_id == 587])

fig = px.line_mapbox(
  trips_gdf_nearest_way[trips_gdf_nearest_way.trip_id == 587], 
  lat="lat", 
  lon="lon", 
  # color="unit_id", 
  hover_data= ['unit_id', 'trip_id','time_stamp'], 
  zoom=9, 
  width=1200, 
  height=800
)

fig.show()

               time_stamp                      unit_id  ...     way_id      dist
70940 2019-01-01 09:45:28  0110003GPSASIA-180500003688  ...  310614155  0.026479
70941 2019-01-01 09:49:19  0110003GPSASIA-180500003688  ...  310614155  0.026479
70942 2019-01-01 09:49:28  0110003GPSASIA-180500003688  ...  310614155  0.026481
70943 2019-01-01 09:53:19  0110003GPSASIA-180500003688  ...  310614155  0.026481
70944 2019-01-01 09:53:30  0110003GPSASIA-180500003688  ...  310614155  0.026482
...                   ...                          ...  ...        ...       ...
71219 2019-01-01 18:11:44  0110003GPSASIA-180500003688  ...  875543258  0.003682
71220 2019-01-01 18:13:52  0110003GPSASIA-180500003688  ...  875543258  0.004783
71221 2019-01-01 18:16:01  0110003GPSASIA-180500003688  ...  875543258  0.000884
71222 2019-01-01 18:16:45  0110003GPSASIA-180500003688  ...  875543258  0.007128
71223 2019-01-01 18:18:09  0110003GPSASIA-180500003688  ...  875543258  0.003130

[284 rows x 10 columns]


In [ ]:
import random
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

drop_dup = trips_gdf_nearest_way[trips_gdf_nearest_way.trip_id == 587].drop_duplicates(subset='way_id')
print(drop_dup)

fig = px.line_mapbox(
  drop_dup, 
  lat="lat", 
  lon="lon", 
  # color="unit_id", 
  hover_data= ['unit_id', 'trip_id','time_stamp'], 
  zoom=9, 
  width=1200, 
  height=800
)

fig.show()

               time_stamp                      unit_id  ...     way_id      dist
70940 2019-01-01 09:45:28  0110003GPSASIA-180500003688  ...  310614155  0.026479
70950 2019-01-01 10:11:31  0110003GPSASIA-180500003688  ...  314705994  0.034151
70956 2019-01-01 10:23:02  0110003GPSASIA-180500003688  ...  545585465  0.023767
70961 2019-01-01 10:30:56  0110003GPSASIA-180500003688  ...  314705989  0.011796
70963 2019-01-01 10:35:57  0110003GPSASIA-180500003688  ...   40268081  0.023452
...                   ...                          ...  ...        ...       ...
71196 2019-01-01 17:33:02  0110003GPSASIA-180500003688  ...  203063309  0.001303
71199 2019-01-01 17:38:05  0110003GPSASIA-180500003688  ...  875543256  0.004676
71207 2019-01-01 17:51:38  0110003GPSASIA-180500003688  ...  875543269  0.001398
71209 2019-01-01 17:55:15  0110003GPSASIA-180500003688  ...  875543258  0.006079
71216 2019-01-01 18:07:26  0110003GPSASIA-180500003688  ...  392807778  0.014845

[119 rows x 10 columns]


In [ ]:
import random
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

passed_way_df = road_df3[road_df3.way_id.isin(trips_gdf_nearest_way[(trips_gdf_nearest_way.trip_id == 587)]['way_id'].unique())]
passed_way_df = passed_way_df.sort_values(by=["way_id", "node_num"])

fig = px.line_mapbox(
  passed_way_df, 
  lat="center_lat", 
  lon="center_lon", 
  color="way_id", 
  hover_data= ['way_id', 'cluster', "center_lat", "center_lon"], 
  zoom=9, 
  width=1200, 
  height=800
)

fig.show()